<a href="https://colab.research.google.com/github/lupis30puc/ultimate_recipe/blob/master/prep_mode_groups_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json #for loading
import pandas as pd
import re #for preprocessing
from gensim.parsing.preprocessing import strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, strip_short
import nltk
from nltk.corpus import stopwords
import en_core_web_sm
import pickle
from google.colab import files
import pickle
import spacy 
from spacy.attrs import ORTH
from collections import Counter
#import textacy
import numpy as np 
import csv

## Loading raw sentences, tokenizer and making the extract:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
raw_sentences = pickle.load(open("/content/drive/My Drive/BPIL_Ultimate Recipe Generator/Colab Notebooks/Pickle files/raw_sentences.pkl", "rb"))
#raw_sentences = pickle.load(open("raw_sentences.pkl", "rb"))

In [ ]:
raw_sentences[0]

'Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover'

In [ ]:
# Load English tokenizer, tagger,  
# parser, NER and word vectors 
nlp = spacy.load("en_core_web_sm")

In [ ]:
print(len(raw_sentences))

341454


In [ ]:
# Selecting and saving only 1000 sentences
extract = raw_sentences[:1000]
print(extract[0])

Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover


# Using latest ingredients list from ingredients team

In [ ]:
# Load the ingredients list
#ingredients_df = pd.read_csv('/content/drive/My Drive/BPIL_Ultimate Recipe Generator/Datasets/ingredient_lemma.csv', index_col=0)
ingredients = pickle.load(open("/content/drive/My Drive/BPIL_Ultimate Recipe Generator/Colab Notebooks/Pickle files/unique_ingredients.p", "rb"))
ingredients.head()

21      salt
20    pepper
23     sugar
31     white
5      onion
Name: Tokens, dtype: object

In [ ]:
#type(ingredients)

pandas.core.series.Series

In [ ]:
ingredients_df = pd.DataFrame({'ingredient': ingredients})
ingredients_df.head()

,ingredient
21,salt
20,pepper
23,sugar
31,white
5,onion


In [ ]:
# Making a list with the ingredients
ingredients = list(ingredients_df.ingredient)
len(ingredients)

934

In [ ]:
# Making a reduced list of less ingredients
ingre_red = ingredients[:]
ingre_red[0]

'salt'

In [ ]:
# Creating a DataFrame with the reduced ingredients list
ingre_df = pd.DataFrame(index=ingre_red)
# Just so there is enough space for the word we add 10000 columns to the DF
for n in range(10000):
  ingre_df[n] = ''

ingre_df = ingre_df.transpose()
ingre_df.head()

,salt,pepper,sugar,white,onion,oil,butter,garlic,flour,egg,cheese,slice,water,milk,olive,vanilla,mince,chicken,green,peel,brown,lemon,tomato,drain,cinnamon,vinegar,soda,beef,bell,potato,advertisement,bread,crush,chocolate,cook,broth,parsley,divide,cheddar,celery,...,rind,idahoan,spiral,plantain,choice,pouch,duncan,tot,dole,citrus,cornflakes,membrane,thread,hines,factory,mangos,crisps,fontina,granola,heinz,skirt,ritz,mexico,chex,hazelnuts,kitchen,grit,seaweed,segments,burgundy,shank,anaheim,zesty,vermicelli,mixture,nestle,shuck,rack,pecorino,buckwheat
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,




> Running the code with the extracted sentences (50000) and all 934 ingredients



In [ ]:
# Loading the filtered original dataset
uploaded=files.upload()

Saving preparation_actions_phases.csv to preparation_actions_phases.csv


In [ ]:
# Load preparations modes:
#prep_mode = pd.read_csv('/content/drive/My Drive/BPIL_Ultimate Recipe Generator/Datasets/Preparation_actions.csv', index_col=0)
prep_mode = pd.read_csv('preparation_actions_phases.csv', delimiter=';')
#prep_mode = prep_mode.transpose()
print(prep_mode.head())

   phase1   phase2  phase3
0  blanch      add    bake
1   carve    chill  braise
2   blend     coat    boil
3    chop  combine    cook
4   clean      dip     fry


In [ ]:
def Union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

prep_modes = Union(list(prep_mode.phase1), list(prep_mode.phase2.dropna())) 
prep_modes_f = Union(prep_modes, list(prep_mode.phase3.dropna()))
print(len(prep_modes_f))
print(prep_modes_f)

89
['blanch', 'saute', 'blend', 'mince', 'wipe', 'clean', 'rinse', 'reduce', 'scale', 'scrub', 'steam', 'add', 'light', 'rub', 'ignite', 'grill', 'crisp', 'refresh', 'spreak', 'splash', 'microwave', 'split', 'pour', 'grease', 'dribble', 'put', 'tenderize', 'whisk', 'carve', 'heat', 'lace', 'dip', 'slit', 'squeeze', 'drain', 'scramble', 'dust', 'top', 'sprinkle', 'roast', 'combine', 'snap', 'weigh', 'cut', 'reheat', 'arrange', 'leave', 'pat', 'stew', 'stir', 'chop', 'chill', 'open', 'cook', 'stir-fry', 'grate', 'boil', 'simmer', 'line', 'ease', 'mix', 'season', 'parboil', 'place', 'warm', 'liquefy', 'slice', 'mash', 'peel', 'slash', 'spray', 'melt', 'dissolve', 'puree', 'soak', 'measure', 'smear', 'fry', 'sift', 'wash', 'bake', 'crush', 'marinate', 'refrigerate', 'salt', 'coat', 'braise', 'preheat', 'knead']


In [ ]:
# We iterate through the list of ingredients and the raw sentences and for each sentences that contains the ingredient we add the verb in it to the DF
for ingre in ingre_red:
  count = 0
  for s in range(len(extract)):
  #for s in extract:
    if ingre in extract[s]:
      doc = nlp(extract[s])
      for token in doc:
        if token.pos_ == 'VERB':
          if str(token) in prep_modes_f:
    #if ingre in filt_sentences[s]:
            try:
              ingre_df[ingre][count] = str(token.lemma_)
              count += 1
            except:
              KeyError
        # melt appears with a space -- 'melt '
ingre_df.head()

,salt,pepper,sugar,white,onion,oil,butter,garlic,flour,egg,cheese,slice,water,milk,olive,vanilla,mince,chicken,green,peel,brown,lemon,tomato,drain,cinnamon,vinegar,soda,beef,bell,potato,advertisement,bread,crush,chocolate,cook,broth,parsley,divide,cheddar,celery,...,rind,idahoan,spiral,plantain,choice,pouch,duncan,tot,dole,citrus,cornflakes,membrane,thread,hines,factory,mangos,crisps,fontina,granola,heinz,skirt,ritz,mexico,chex,hazelnuts,kitchen,grit,seaweed,segments,burgundy,shank,anaheim,zesty,vermicelli,mixture,nestle,shuck,rack,pecorino,buckwheat
0,combine,combine,mix,combine,mix,add,mix,add,combine,stir,stir,add,mix,cook,add,stir,,combine,stir,,cook,mix,stir,drain,whisk,combine,combine,add,,add,,mix,,stir,cook,stir,combine,,,combine,...,combine,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,stir,,,mix,,
1,add,mix,add,sprinkle,stir,cook,mix,combine,stir,mix,sprinkle,arrange,add,mash,cook,stir,,stir,drain,,mix,stir,combine,cook,mix,whisk,whisk,drain,,cook,,combine,,pour,place,reduce,combine,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,cook,,,combine,,
2,cook,drain,stir,stir,add,add,melt,mix,cook,add,mix,,cook,add,stir,mix,,cook,melt,,mix,,stir,drain,combine,stir,combine,combine,,mash,,mix,,,mix,add,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,reduce,,,pour,,
3,add,mix,combine,mix,cook,mix,cook,mix,whisk,combine,combine,,mix,stir,stir,combine,,simmer,saute,,cook,,combine,drain,sprinkle,stir,mix,mix,,cook,,combine,,,add,add,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,cook,,,,,
4,whisk,place,sift,mix,mix,add,pour,cook,combine,stir,sprinkle,,add,cook,combine,stir,,pour,stir,,cook,,,drain,sprinkle,add,stir,saute,,simmer,,mix,,,cook,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,stir,,,,,


In [ ]:
ingre_df.describe()

,salt,pepper,sugar,white,onion,oil,butter,garlic,flour,egg,cheese,slice,water,milk,olive,vanilla,mince,chicken,green,peel,brown,lemon,tomato,drain,cinnamon,vinegar,soda,beef,bell,potato,advertisement,bread,crush,chocolate,cook,broth,parsley,divide,cheddar,celery,...,rind,idahoan,spiral,plantain,choice,pouch,duncan,tot,dole,citrus,cornflakes,membrane,thread,hines,factory,mangos,crisps,fontina,granola,heinz,skirt,ritz,mexico,chex,hazelnuts,kitchen,grit,seaweed,segments,burgundy,shank,anaheim,zesty,vermicelli,mixture,nestle,shuck,rack,pecorino,buckwheat
count,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
unique,14,9,10,7,11,13,10,8,14,8,9,3,10,9,5,4,1,11,5,1,9,3,3,4,6,5,5,7,1,5,1,3,1,3,11,4,2,1,1,2,...,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,11,1,1,4,1,1
top,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
freq,9964,9982,9965,9992,9976,9973,9977,9981,9966,9978,9981,9998,9979,9978,9995,9990,10000,9984,9995,10000,9980,9998,9996,9992,9990,9994,9988,9990,10000,9993,10000,9994,10000,9998,9946,9996,9998,10000,10000,9999,...,9999,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,9965,10000,10000,9997,10000,10000


-- Count verbs per ingredient and see percentages
-- Return the 3 highest percentages (most likely prep modes)


In [ ]:
# Counting frequencies of prep modes per ingredient, and making a dictionary with the top 4
from heapq import nlargest
#ingre2_df.nlargest(3, ingre)
#(df.iloc[:3])
ingre_top_modes = {}
for ingre in ingre_red:
  top4 = ingre_df[ingre].value_counts().nlargest(11)
  ingre_top_modes[ingre] = top4.to_dict()

print(ingre_top_modes['garlic'])

{'': 9981, 'mix': 5, 'stir': 4, 'cook': 3, 'combine': 3, 'add': 2, 'simmer': 1, 'salt': 1}


In [ ]:
# remove the top 1 = '', get only the prep modes!
for ingre in ingre_red:
  if '' in ingre_top_modes[ingre]: 
    del ingre_top_modes[ingre]['']
print(ingre_top_modes['garlic'])

{'mix': 5, 'stir': 4, 'cook': 3, 'combine': 3, 'add': 2, 'simmer': 1, 'salt': 1}


In [ ]:
#pickle.dump(ingre_top_modes, open("ingredients_prep_modes_50000_2.pkl", "wb"))

-- With a function and the final ingredients list search in our ingre_prep_mode file and return the different preparation modes
--> done in separate notebook

In [ ]:
print(len(ingre_top_modes))

934


In [ ]:
for ingre in ingre_red:
  if len(ingre_top_modes[ingre]) == 0: 
    del ingre_top_modes[ingre]
print(len(ingre_top_modes))

268


In [ ]:
#ingre_top_modes.keys()

In [ ]:
"""ordered = None
ingre_top_sort = ingre_top_modes.copy()
for ingre in ingre_red:
  try:
    ordered = {k: v for k, v in sorted(ingre_top_modes[ingre].items(), key=lambda item: item[1], reverse=True)}
    ingre_top_sort[ingre] = ordered
  except:
    KeyError
ingre_top_modes[ingre] = ordered
print(ingre_top_modes['eggs'])"""

{'stir': 5, 'mix': 4, 'whisk': 1, 'combine': 1, 'add': 1}


In [ ]:
#att = ingre_top_modes['eggs']
#print(att)

{'stir': 5, 'mix': 4, 'whisk': 1, 'combine': 1, 'add': 1}


In [ ]:
"""
from collections import OrderedDict
ord_eggs = OrderedDict(sorted(ingre_top_modes['eggs'].items(), key=lambda x: x[1], reverse=True))
print(ord_eggs)
"""

OrderedDict([('stir', 5), ('mix', 4), ('whisk', 1), ('combine', 1), ('add', 1)])


In [ ]:
phase1_modes = list(prep_mode.phase1)
phase2_modes = list(prep_mode.phase2.dropna())
phase3_modes = list(prep_mode.phase3.dropna())
print(len(phase1_modes))
print(phase3_modes)

48
['bake', 'braise', 'boil', 'cook', 'fry', 'grill', 'heat', 'ignite', 'microwave', 'roast', 'scramble', 'simmer', 'steam', 'stew', 'stir-fry', 'warm', 'arrange', 'crisp', 'dribble']


In [ ]:
# a dict of dicts with ingredient and top modes


new_dict = {}


if ingre.mode is in phase 1 or 2:
  ordered = {k: v for k, v in sorted(ingre_top_modes[ingre].items(), key=lambda item: item[1], reverse=True)}
  previous.phase = ingre.mode.phase
  add ingre to new_dict
elif ingre.mode =! previous.phase:
  add ingred
  pass

# something with verbs of different phases for each ingredient.
# a dataframe
# ----    phase 1 | phase 2 | phase 3
# ingre1 | cut    | mix     | fry

In [ ]:
att = ingre_top_modes['beef']
print(att)
# phase 2: combine, add , mix, saute
# phase 1: drain
# phase 3: heat

# ideal result: beef: {combine, heat}

{'combine': 3, 'add': 2, 'mix': 2, 'heat': 1, 'saute': 1, 'drain': 1}


In [ ]:
# selecting modes from different phases for each ingredient

ingre_modes_order = {}
for ingre in list(ingre_top_modes.keys()):
  modes_order = []
  ordered = {k: v for k, v in sorted(ingre_top_modes[ingre].items(), key=lambda item: item[1], reverse=True)}
  # go through all the ordered modes
  actual_phase = None
  for mode in list(ordered.keys()):
    if mode in phase1_modes and actual_phase != 1 and actual_phase != 3:
      actual_phase = 1
      modes_order.append(mode)
    elif mode in phase2_modes and actual_phase != 2 and actual_phase != 3:
      actual_phase = 2
      modes_order.append(mode)
    elif mode in phase3_modes and actual_phase != 3:
      actual_phase = 3
      modes_order.append(mode)
  
  ingre_modes_order[ingre] = modes_order
  
print(ingre_modes_order['pepper'])

['mix', 'pour', 'coat', 'drain']


In [ ]:
# selecting modes from different phases for each ingredient

ingre_modes_order = {}
for ingre in list(ingre_top_modes.keys()):
  modes_order = []
  ordered = {k: v for k, v in sorted(ingre_top_modes[ingre].items(), key=lambda item: item[1], reverse=True)}
  # go through all the ordered modes
  phase1 = []
  phase2 = []
  phase3 = []
  for mode in list(ordered.keys()):
    if mode in phase1_modes:
      phase1.append(mode)
    elif mode in phase2_modes:
      phase2.append(mode)
    elif mode in phase3_modes:
      phase3.append(mode)
  #print(ordered)
  #print(phase1)
  #print(phase2)
  #print(phase3)
  try:
    modes_order.append(phase1[0])
    modes_order.append(phase2[0])
    modes_order.append(phase3[0])
  except:
    IndexError

  ingre_modes_order[ingre] = modes_order

print(ingre_modes_order['pepper'])

['pour', 'mix']


In [ ]:
print(ingre_modes_order['garlic'])

[]


In [ ]:
# insert modes for each ingredient for which phases doesn't work!

#ingre_modes_order = {}
for ingre in list(ingre_top_modes.keys()):
  modes_order = []
  ordered = {k: v for k, v in sorted(ingre_top_modes[ingre].items(), key=lambda item: item[1], reverse=True)}
  # go through all the ordered modes
  m_li = list(ordered.keys())
  try:
    modes_order.append(m_li[0])
    modes_order.append(m_li[1])
  except:
    IndexError

  ingre_modes_order[ingre] = modes_order

print(ingre_modes_order['garlic'])

['mix', 'stir']


In [ ]:
ingre_modes_order

{'add': ['add', 'stir'],
 'almond': ['stir', 'combine'],
 'almonds': ['stir', 'combine'],
 'apple': ['sprinkle', 'mix'],
 'avocado': ['mash'],
 'avocados': ['mash'],
 'bacon': ['add'],
 'baguette': ['arrange'],
 'baking': ['stir', 'place'],
 'ball': ['place'],
 'banana': ['stir'],
 'bar': ['stir', 'mix'],
 'barbecue': ['mix'],
 'barbeque': ['stir'],
 'basil': ['combine'],
 'bay': ['add'],
 'bean': ['simmer', 'mix'],
 'beans': ['simmer', 'mix'],
 'beat': ['add', 'cook'],
 'beef': ['combine', 'add'],
 'beer': ['pour'],
 'biscuit': ['place'],
 'blend': ['stir', 'blend'],
 'boil': ['cook', 'boil'],
 'boiling': ['stir', 'add'],
 'bouillon': ['mix'],
 'bow': ['mix', 'combine'],
 'bread': ['combine', 'mix'],
 'broccoli': ['stir'],
 'broth': ['add', 'stir'],
 'brown': ['cook', 'mix'],
 'bun': ['pour'],
 'butt': ['stir', 'mix'],
 'butter': ['stir', 'mix'],
 'buttermilk': ['stir'],
 'cake': ['stir', 'bake'],
 'caraway': ['combine'],
 'carrot': ['combine'],
 'carrots': ['combine'],
 'cauliflower'

In [ ]:
# taking garlic as an example

gar_modes_order = {}
gmodes_order = []
garlic = {k: v for k, v in sorted(ingre_top_modes['garlic'].items(), key=lambda item: item[1], reverse=True)}
# go through all the ordered modes
p1 = []
p2 = []
p3 = []
g_li = list(garlic.keys())
for m in range(len(g_li)):
  print(g_li[m])
  if g_li[m] in phase1_modes:
    phase1.append(g_li[m])
  elif mode in phase2_modes:
    phase2.append(g_li[m])
  elif mode in phase3_modes:
    phase3.append(g_li[m])
print(garlic)
print(p1)
print(p2)
print(p3)
try:
  #modes_order.append(p1[0])
  modes_order.append(p2[0])
  modes_order.append(p3[0])
except:
  IndexError

gar_modes_order['garlic'] = gmodes_order

print(gar_modes_order['garlic'])

mix
stir
cook
combine
add
simmer
salt
{'mix': 5, 'stir': 4, 'cook': 3, 'combine': 3, 'add': 2, 'simmer': 1, 'salt': 1}
[]
[]
[]
[]


In [ ]:
phase2_modes

['add',
 'chill',
 'coat',
 'combine',
 'dip',
 'dust',
 'marinate',
 'mash',
 'mix',
 'parboil',
 'place',
 'preheat',
 'put',
 'puree',
 'salt',
 'saute',
 'season',
 'soak',
 'sprinkle',
 'stir',
 'top',
 'whisk']

In [ ]:
print(garlic.keys())

dict_keys(['mix', 'stir', 'cook', 'combine', 'add', 'simmer', 'salt'])


In [ ]:
print(ingre_top_modes['beef'].keys())

dict_keys(['combine', 'add', 'mix', 'heat', 'drain', 'saute'])


In [ ]:
print(ingre_top_modes['garlic'])
# phase 1: drain, pour
# phase 2: mix, stir, combine, coat, place
# phase 3: --

# ideal result: pepper: {mix, stir}

{'mix': 5, 'stir': 4, 'cook': 3, 'combine': 3, 'add': 2, 'simmer': 1, 'salt': 1}
